In [1]:
import sys
import numpy as np
sys.path.append("/SNS/users/m2d/git/machine_learning/src")
import importlib
import models
import os
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

2023-06-11 18:07:41.003424: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
data_dir = os.path.expanduser("~m2d/git/analysis_playground/ml/data/training_data")

pars = np.load(os.path.join(data_dir, "orhs3-noise_pars.npy"))[:1000000]
refl = np.load(os.path.join(data_dir, "orhs3-noise_data.npy"))[:1000000]
q_values = np.load(os.path.join(data_dir, "orhs3_q_values.npy"))

print(pars.shape)
print(refl.shape)
print(q_values.shape)

idx = 0
nset=10000


(1000000,)
(1000000, 100)
(100,)


In [7]:
print(pars)

[1. 2. 3. ... 2. 1. 2.]


In [3]:
onelayerpars= []
onelayerrefl= []
twolayerpars= []
twolayerrefl= []
threelayerpars= []
threelayerrefl= []

for i in range ( len(pars)-1 ):
    if pars [i]==1:
        onelayerpars.append(pars[i])
        onelayerrefl.append(refl[i])
    if pars [i]==2:
        twolayerpars.append(pars[i])
        twolayerrefl.append(refl[i])
    else:
        threelayerpars.append(pars[i])
        threelayerrefl.append(refl[i])
                       
onepars=np.asarray(onelayerpars)
onerefl=np.asarray(onelayerrefl)
twopars=np.asarray(twolayerpars)
tworefl=np.asarray(twolayerrefl)
threepars=np.asarray(threelayerpars)
threerefl=np.asarray(threelayerrefl)

In [4]:
refl= np.asarray(threerefl[:-nset])
pars= np.asarray (threepars[:-nset])

testset= np.asarray(threerefl[-nset:])
testsetout= np.asarray(threepars[-nset:])
#testpars = np.asarray (threepars[-nset:])

In [5]:
import importlib
importlib.reload(models)
n_data = refl.shape[0]
n_features = refl.shape[1]
n_outputs = 9

encoder=models.cnn_encoder(n_data, n_features, n_outputs)
encoder.compile(loss='mean_squared_error' , optimizer=tf.keras.optimizers.Adam())
history = encoder.fit(refl, pars,
                        epochs=20, batch_size=256,
                        validation_data=(testset, testset))

2023-06-11 12:26:50.798331: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/20
2565/2565 [==============================] - ETA: 0s - loss: 0.3748

ValueError: in user code:

    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/training.py", line 1852, in test_function  *
        return step_function(self, iterator)
    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/training.py", line 1836, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/training.py", line 1824, in run_step  **
        outputs = model.test_step(data)
    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/training.py", line 1790, in test_step
        self.compute_loss(x, y, y_pred, sample_weight)
    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/opt/anaconda/envs/tensorflow/lib/python3.11/site-packages/keras/engine/compile_utils.py", line

In [ ]:
fig = plt.figure(dpi=100, figsize=[8,4])

plt.plot(np.arange(len(history.history['loss'])), history.history['loss'], label='loss')
plt.plot(np.arange(len(history.history['val_loss'])), history.history['val_loss'], label='validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.yscale('log')
plt.xscale('linear')
plt.show()

